In [ ]:
import tensorflow as tf
import pandas as pd
import requests 
import json
import base64

from typing import Dict

In [ ]:
def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
          value=[value]  
        ),
    )
    
def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(
          value=[value]  
        ),
    )

In [ ]:
def prepare_json(inputs: Dict):
    feature_spec = {}
    
    for key, value in inputs.items():
        if isinstance(value, float):
            feature_spec[key] = float_feature(value)
        else:    
            feature_spec[key] = int_feature(value)
        
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({"instances": result})

In [ ]:
def make_prediction(inputs: Dict):
    json_data = prepare_json(inputs)
    
    endpoint = "http://localhost:8080/v1/models/movie-cf-model:predict"
    response = requests.post(endpoint, data=json_data)
    
    prediction = response.json()["predictions"][0][0]
    
    return f"Predict movie rating with Movie Id {inputs['movieId']} and User Id {inputs['userId']}: {prediction}"

In [ ]:
make_prediction({"movieId": 42, "userId": 123})